### Data Extraction from XML

In [2]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
tree = ET.parse('train_data.xml')
reviews = tree.getroot()




In [3]:

Sentence_array=list()
asp_array=list()
ref_revarray=list()
effect_array=list()
revindex=list()
i=0
j=0
prev=0
num_sent=0

polar = {
    "negative": 0.0,
    "neutral":0.5,
    "positive":1.0
}

In [4]:
for review in reviews:

    if(i==299):
        ref_revarray.append(i)
        num_sent+=1
        effect_array.append(0.0)
        asp_array.append('FOOD#QUALITY')
        Sentence_array.append("Food is very bad")

    sentences = review.find("./sentences")
    for sentence in sentences :
        num_sent=num_sent+1
        text = sentence.find("./text")
        opinions = sentence.findall("./Opinions/Opinion")
        for opinion in opinions :
            if(prev == i):
                j=j+1
            else:
                revindex.append(j)
                j=1
                prev=i

            effect_array.append(polar[opinion.attrib['polarity']])
            Sentence_array.append(text.text)
            asp_array.append(opinion.attrib['category'])
            ref_revarray.append(i)

    i=i+1

revindex=np.array(revindex)
pol_arr=np.array(effect_array)

maxi=np.max(revindex)

print("Total number of sentences are " + str(len(Sentence_array)))
print("Maximim number of sentences in a review is " + str(maxi))



print(num_sent)  
print(len(Sentence_array))
print(len(asp_array))
print(len(ref_revarray))
print(len(effect_array))




Total number of sentences are 2508
Maximim number of sentences in a review is 44
2001
2508
2508
2508
2508


Till now we have extracted Info from XML, From now on let us create a dataframe out of them right now

### introducing pandas

In [5]:
data=list()

for i in range(len(Sentence_array)):
  data.append([ref_revarray[i], asp_array[i],Sentence_array[i],effect_array[i]])
  
# Create the pandas DataFrame
DATA = pd.DataFrame(data, columns = ['rid','aspect','text','polarity'])

DATA[0:150]


,rid,aspect,text,polarity
0,0,RESTAURANT#GENERAL,Judging from previous posts this used to be a ...,0.0
1,0,SERVICE#GENERAL,"We, there were four of us, arrived at noon - t...",0.0
2,0,SERVICE#GENERAL,"They never brought us complimentary noodles, i...",0.0
3,0,FOOD#QUALITY,The food was lousy - too sweet or too salty an...,0.0
4,0,FOOD#STYLE_OPTIONS,The food was lousy - too sweet or too salty an...,0.0
...,...,...,...,...
145,22,AMBIENCE#GENERAL,"Leon is an East Village gem: casual but hip, w...",1.0
146,22,FOOD#QUALITY,"Leon is an East Village gem: casual but hip, w...",1.0
147,22,AMBIENCE#GENERAL,"Leon is an East Village gem: casual but hip, w...",1.0
148,22,FOOD#QUALITY,"Leon is an East Village gem: casual but hip, w...",1.0


**Done, Now let us clean the text using standard methods avaliable for US:XD**

### Cleaning Step:  Removing Links, HTML, Emoji, Punctuation

In [6]:
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))


def rem_stopwords(text):
    text1 = [w for w in text.split() if not w.lower() in stop_words]
    return " ".join(text1)

def tokenise_data(samp_text):
    word_tokens = [word.lower() for word in word_tokenize(samp_text)]
    return word_tokens
    



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sriramprasad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import re
import string


def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)


def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)

def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)

def remove_punct(text):
  table = str.maketrans("", "", string.punctuation)
  return text.translate(table)



In [8]:
import sys
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install tensorflow

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


2022-12-02 22:35:08.812125: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 22:35:12.272060: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-02 22:35:13.237157: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-02 22:35:13.237204: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

We have to remove stop words in the sentences for making it more efficient

In [9]:
DATA.text = DATA.text.map(lambda x: remove_URL(x))
DATA.text = DATA.text.map(lambda x: remove_html(x))
DATA.text = DATA.text.map(lambda x: remove_emoji(x))
DATA.text = DATA.text.map(lambda x: remove_punct(x))
DATA.text = DATA.text.map(rem_stopwords)

hell=list()

corpus=list()
size=list()


hell = DATA.loc[:,"text"].tolist()
hell1 = DATA.loc[:,"rid"].tolist()

for i in range(len(Sentence_array)):
    corpus.append(tokenise_data(hell[i]))
    size.append(len(corpus[i]))


corpus=np.array(corpus)
hell1 = np.array(hell1)
size=np.array(size)

maximum= np.max(size)
maximum1=np.max(hell1)
print("maximum words in a sentence are " + str(maximum))
print("maximum sentences in a review are" +str(maxi))
print("Total Number Of Reviews",+maximum1+1)


maximum words in a sentence are 35
maximum sentences in a review are44
Total Number Of Reviews 350


/tmp/ipykernel_11204/2397471873.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  corpus=np.array(corpus)


**Recap of What we have done till now:**


1. Extracted all the relevant information from the xml file
2. Created a pandas dataframe out of the required data for making our lives simpler
3. Cleaned the data from pincuations, links, emojis, html   *italicized text*

**Now let us create word embeddings for proceeding with the implementaion**



### Tokenizing, Padding, Preparing Deliverables for embedding Matrix 

In [10]:
tokenizer = Tokenizer(num_words=1000000)
tokenizer.fit_on_texts(DATA.text)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(DATA.text)

paddedsequences = pad_sequences(
sequences, maxlen=35, truncating="post", padding="post"
)

print(paddedsequences)
print(paddedsequences)
print(type(paddedsequences[0]))
print(len(paddedsequences[0]))


[[2058  609 2059 ...    0    0    0]
 [ 202   34  516 ...    0    0    0]
 [  17  611   34 ...    0    0    0]
 ...
 [ 605  347  292 ...    0    0    0]
 [ 249   91    7 ...    0    0    0]
 [ 249   91    7 ...    0    0    0]]
[[2058  609 2059 ...    0    0    0]
 [ 202   34  516 ...    0    0    0]
 [  17  611   34 ...    0    0    0]
 ...
 [ 605  347  292 ...    0    0    0]
 [ 249   91    7 ...    0    0    0]
 [ 249   91    7 ...    0    0    0]]
<class 'numpy.ndarray'>
35


Now let us create train sequences which can be processed with embeddings for catching the relation better

**IMPORTANT OBSERVATIONS:<br>
1) MAXIMUM NUMBER OF WORDS IN A SENTENCE IS : 35 <br>
2) MAXIMUM NUMBER OF SENTENCES IN A REVIEW IS : 44<br>
3) TOTAL NUMBER OF REVIEWS : 350**

In [11]:
newpaddingsequences = []
newpolarity = []
newaspect= []
newrid=[]
temp=[]
import numpy as np
padding = np.zeros((35))

j=0
for i in range(349):

  limit44 = 0
  while(ref_revarray[j]==i):
    newpaddingsequences.append(paddedsequences[j])
    newpolarity.append(effect_array[j])
    newaspect.append(asp_array[j])
    newrid.append(i)

    limit44+=1
    j+=1
    #print(j)
    #print(i)
  while(limit44 < 44):
    newpaddingsequences.append(padding)
    newpolarity.append(0.5)
    newaspect.append("FOOD#QUALITY")
    newrid.append(i)
    limit44 +=1

# print(len(newpaddingsequences))
# print(len(newpolarity[:44]))
newpaddingsequences = np.array(newpaddingsequences)
newpolarity = np.array(newpolarity)
newaspect = np.array(newaspect)
newrid = np.array(newrid)
temp = np.array(temp)
new_aspect_array=[]
for i in range(len(asp_array)):
    new_aspect_array.append(((asp_array[i]).split('#')))
    new_aspect_array[i][0]=new_aspect_array[i][0].lower()
    new_aspect_array[i][1]=new_aspect_array[i][1].lower()
    if(new_aspect_array[i][1]=='style_options'):
        new_aspect_array[i][1]='style'
#         print("yes")

# print((new_aspect_array))
data = []
for i in range(len(newpaddingsequences)):
  data.append([newrid[i], newaspect[i],newpaddingsequences[i],newpolarity[i]])
  
# Create the pandas DataFrame
DATA = pd.DataFrame(data, columns = ['rid','aspect','sequence','polarity'])


DATA.sequence



0        [2058.0, 609.0, 2059.0, 843.0, 4.0, 5.0, 610.0...
1        [202.0, 34.0, 516.0, 2060.0, 5.0, 411.0, 11.0,...
2        [17.0, 611.0, 34.0, 612.0, 517.0, 1417.0, 2063...
3        [1.0, 845.0, 258.0, 613.0, 162.0, 343.0, 0.0, ...
4        [1.0, 845.0, 258.0, 613.0, 162.0, 343.0, 0.0, ...
                               ...                        
15351    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
15352    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
15353    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
15354    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
15355    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: sequence, Length: 15356, dtype: object

This sequencing and stuff is done, now let us go to word embedddings baby

In [12]:
final_train_seq=paddedsequences[:2010]
final_test_seq=paddedsequences[2010:]

final_train_label=pol_arr[:2010]
final_test_label=pol_arr[2010:]

print(final_train_seq.shape)

print(final_test_seq)


(2010, 35)
[[2821  647    0 ...    0    0    0]
 [   2    1    0 ...    0    0    0]
 [  51  396  519 ...    0    0    0]
 ...
 [ 605  347  292 ...    0    0    0]
 [ 249   91    7 ...    0    0    0]
 [ 249   91    7 ...    0    0    0]]


### Embeddings

In [13]:
import numpy as np
embedding_dict = {}
with open("glove.twitter.27B.100d.txt", "r",encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], "float32")
        embedding_dict[word] = vectors
f.close()

In [14]:
embedding_dict['penbgt']

array([-0.65304  ,  0.67496  , -0.52621  ,  0.30145  ,  0.069593 ,
       -0.15635  ,  0.053281 , -0.76136  ,  0.79736  , -0.41488  ,
        0.60945  ,  0.14195  ,  1.337    ,  0.24787  ,  1.144    ,
        0.6167   , -0.28845  ,  0.32831  ,  1.2722   ,  0.42938  ,
        1.4665   ,  0.048341 , -0.20411  ,  0.36426  , -0.16438  ,
        0.86703  ,  0.14129  ,  0.76295  , -0.8172   ,  0.11017  ,
        0.50265  ,  0.13047  ,  1.0354   , -1.0856   , -0.30595  ,
        0.34106  ,  0.14021  , -0.027409 , -0.24942  ,  0.25416  ,
        0.64679  , -0.22021  ,  0.39994  ,  0.3335   , -0.13457  ,
        0.01284  , -0.85238  , -0.22307  ,  0.59499  ,  0.73318  ,
        1.1219   ,  0.30758  ,  0.58002  ,  0.42044  ,  0.43146  ,
       -0.0503   ,  0.5681   ,  0.011106 ,  0.34734  , -0.15787  ,
       -0.071503 , -0.016518 ,  0.93803  , -0.74437  ,  0.1035   ,
       -0.58374  ,  0.14542  , -0.34447  ,  0.47532  , -0.22283  ,
       -0.49388  ,  0.27964  , -0.16178  , -0.55446  , -0.0064

In [15]:
num_words = len(word_index) +1
embedding_matrix = np.zeros((num_words, 100))

for word, i in word_index.items():
    if i < num_words:
        emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec
word_index
print(embedding_matrix)

embedding_dict.get('food')

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.52348    -0.39926001  0.24898    ... -0.023053   -0.019998
   0.2467    ]
 [ 0.1073      0.36058     0.11135    ...  0.14933001 -0.30864999
  -0.55198002]
 ...
 [-0.42250001 -0.36368999 -0.33282    ... -0.74137002  0.15254
  -0.17957   ]
 [-0.28277001  0.71575999  0.44635001 ...  0.14325     1.02569997
   0.37064999]
 [-0.14717001 -0.094362   -0.1002     ...  0.074088   -0.15397
  -0.18822999]]


array([-0.52348  , -0.39926  ,  0.24898  , -0.039715 ,  0.067344 ,
       -0.34445  ,  0.34888  ,  0.60016  ,  0.94921  , -0.64628  ,
       -1.1042   , -0.44242  , -4.2723   ,  0.47759  , -0.54165  ,
        0.55117  ,  0.26203  , -0.47045  , -0.37717  , -0.5985   ,
       -0.10843  ,  0.32041  , -0.40699  ,  0.004276 ,  0.5429   ,
        0.42808  , -0.16087  , -0.13919  ,  0.58994  ,  0.1809   ,
       -0.88564  , -0.047295 , -0.97389  , -0.0083825,  0.085801 ,
       -0.68881  ,  0.41937  ,  0.73735  ,  0.69426  ,  0.97459  ,
       -0.43348  ,  0.33125  ,  0.065084 , -0.13284  ,  0.99607  ,
        0.82727  ,  0.065382 ,  0.80963  , -0.095502 ,  0.049895 ,
       -0.77063  , -0.24953  , -0.76081  ,  0.91922  ,  0.20773  ,
        0.41041  ,  0.71853  , -0.22399  ,  0.2767   , -0.44373  ,
       -0.35005  , -0.15431  ,  0.013649 ,  0.38956  ,  0.32611  ,
       -0.84561  ,  0.27346  , -0.486    , -0.17267  , -0.081096 ,
       -0.30867  , -0.20255  ,  0.33779  ,  0.27343  ,  0.4622

In [16]:
print((embedding_matrix.shape))

(3053, 100)


In [17]:
aa=new_aspect_array[0][0]

embedding_dict.get(aa)
new_aspect_array[0][0]

'restaurant'

In [18]:
print(final_train_seq)
print(final_train_label)
num_words
type(final_train_label)

[[2058  609 2059 ...    0    0    0]
 [ 202   34  516 ...    0    0    0]
 [  17  611   34 ...    0    0    0]
 ...
 [   8  371  555 ...    0    0    0]
 [ 279   55    8 ...    0    0    0]
 [  37    1    0 ...    0    0    0]]
[0. 0. 0. ... 1. 1. 1.]


numpy.ndarray

In [19]:

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Input
from keras.initializers import Constant
from tensorflow.keras.optimizers import Adam

model = Sequential()

model.add(
    Embedding(
        num_words,
        100,
        embeddings_initializer=Constant(embedding_matrix),
        input_length=35,
        trainable=False,
    )
)
model.add(Bidirectional(LSTM(15,dropout=0.5)))
model.add(Dense(1, activation="sigmoid"))



optimizer = Adam(learning_rate=3e-4)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.summary

2022-12-02 22:36:10.704637: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-02 22:36:10.704804: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sriram): /proc/driver/nvidia/version does not exist
2022-12-02 22:36:10.875247: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x7feafc05a460>>

In [20]:
print(final_train_seq.shape)
print(final_train_label.shape)



(2010, 35)
(2010,)


In [21]:
history = model.fit(
    final_train_seq,
    final_train_label,
    epochs=20,
    validation_data=(final_train_seq,final_train_label),
    verbose=1,
)

Epoch 1/20
63/63 [==============================] - 10s 43ms/step - loss: 0.6418 - accuracy: 0.6622 - val_loss: 0.5857 - val_accuracy: 0.7204
Epoch 2/20
63/63 [==============================] - 2s 28ms/step - loss: 0.5705 - accuracy: 0.7209 - val_loss: 0.5481 - val_accuracy: 0.7204
Epoch 3/20
63/63 [==============================] - 2s 32ms/step - loss: 0.5219 - accuracy: 0.7204 - val_loss: 0.4788 - val_accuracy: 0.7214
Epoch 4/20
63/63 [==============================] - 2s 29ms/step - loss: 0.4810 - accuracy: 0.7403 - val_loss: 0.4445 - val_accuracy: 0.7418
Epoch 5/20
63/63 [==============================] - 2s 28ms/step - loss: 0.4705 - accuracy: 0.7562 - val_loss: 0.4365 - val_accuracy: 0.7836
Epoch 6/20
63/63 [==============================] - 2s 29ms/step - loss: 0.4625 - accuracy: 0.7766 - val_loss: 0.4223 - val_accuracy: 0.7826
Epoch 7/20
63/63 [==============================] - 2s 29ms/step - loss: 0.4450 - accuracy: 0.7721 - val_loss: 0.4107 - val_accuracy: 0.8095
Epoch 8/20
6

In [22]:
print(final_test_seq.shape,final_test_label.shape)

(498, 35) (498,)


In [23]:
pred = model.predict(final_test_seq)
print(pred.shape)
print(pred[0])
pred_int=[]

pred_int = pred.round().astype("int")
pred_int


16/16 [==============================] - 1s 8ms/step
(498, 1)
[0.8866892]


array([[1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
    

In [24]:
# print(final_test_sequences)
print(len(final_test_label))
ccc=0
for i in range(len(final_test_label)):
    if(pred_int[i]==final_test_label[i]):
#         print(pred_int[i])
        ccc+=1
print(ccc)

498
355


In [26]:
rev_polarity=[]
j=0
for j in range(len(revindex)):
    sum_1=0
    count_review=0
    average=0
    for i in range(len(ref_revarray)):
        if(ref_revarray[i]==j):
            count_review+=1
            sum_1+=pol_arr[i]
    average=sum_1/count_review
    average=average.round().astype("int")
    rev_polarity.append(average)

print(np.shape(rev_polarity))

(348,)


In [27]:
found_polarity=[]
# print(type(pred_int[0]))
j=0
# print(len(pred))
for j in range(len(revindex)):
    sum_1=0
    count_review=0
    average=0
    for i in range(len(pred)):

        if(ref_revarray[2010+i]==j):
            count_review+=1
            sum_1+=pred_int[i]
    if(count_review==0):
        continue
    average=sum_1/count_review
    average=average.round().astype("int")
    found_polarity.append(average)

print(np.shape(found_polarity))
len(found_polarity)

(43, 1)


43

In [34]:
count=0
aa=(ref_revarray[2010])
#print(int(0.75*len(found_polarity)))
for i in range(int(0.85*len(found_polarity))):
    if(found_polarity[i][0]==rev_polarity[aa+i]):
        count+=1
#print(count)
print(count/len(found_polarity)*100,"%")

67.44186046511628 %


In [38]:
entity_matrix=[]
attribute_matrix=[]
aspect_embedding=[]

for i in range(len(new_aspect_array)):
    entity=new_aspect_array[i][0]
    entity_matrix=embedding_dict.get(entity)

    attribute=new_aspect_array[i][1]
    attribute_matrix=embedding_dict.get(attribute)

    aspect_embedding.append((attribute_matrix+entity_matrix)/2)

aspect_embedding = np.array(aspect_embedding)
print(aspect_embedding.shape)
print(len(aspect_embedding))

(2508, 100)
2508


In [41]:
akidea  = model.predict(final_train_seq)
print(akidea.shape)
print(akidea[0])
akidea_int=[]

akidea_int = akidea.round().astype("int")

x = float(akidea[0])
x

63/63 [==============================] - 1s 7ms/step
(2010, 1)
[0.27173406]


0.2717340588569641

In [42]:
from array import *
step_train_sequences=[]
step_train_labels=[]

for i in range (len(akidea)):
  temp=[]
  for j in range (len(aspect_embedding[i])):

    temp.append(float(aspect_embedding[i][j]))
     
  temp.append(float(akidea[i]))
  step_train_sequences.append(temp)
  step_train_labels.append((effect_array[i]))


import random

for i in range(len(step_train_labels)):
  if(step_train_labels[i]==0.5):
    zendaya= random.randint(0,9)
    if(zendaya < 5):
      step_train_labels[i] = 0.0
    else:
      step_train_labels[i] = 1.0
print(step_train_sequences[0])
np.shape(step_train_sequences)

[0.09622500091791153, 0.4270249903202057, -0.11979099363088608, -0.19187000393867493, 0.3706650137901306, -0.28080499172210693, -0.23441749811172485, 0.059316497296094894, 0.205035001039505, -0.6713050007820129, -0.6455274820327759, -0.15425999462604523, -2.578050136566162, 0.1284799873828888, 0.3696550130844116, 0.26439449191093445, -0.10473499447107315, 0.09650349617004395, -0.19606000185012817, -0.21740999817848206, 0.3145400285720825, -0.38535499572753906, -0.2544499933719635, -0.7935900092124939, 0.07619500160217285, -0.8403499722480774, 0.30768999457359314, -0.12316499650478363, 0.1751599907875061, -0.07569500058889389, 0.12114900350570679, -0.37713998556137085, -0.8111000061035156, -0.6666949987411499, 0.3620949983596802, -0.612089991569519, 0.2086549997329712, 1.0045100450515747, 0.30751001834869385, -0.23152700066566467, 0.08579500019550323, 0.19830000400543213, 0.19345498085021973, 0.14892999827861786, 0.3829195201396942, 0.40740498900413513, 0.38093000650405884, 0.2786349952

(2010, 101)

In [44]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Input
from keras.initializers import Constant
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

rev_model = Sequential()
model.add(
    Embedding(
        num_words,
        100,
        embeddings_initializer=Constant(embedding_matrix),
        input_length=101,
        trainable=False,
    )
)
rev_model.add(Bidirectional(LSTM(15,dropout=0.5)))
rev_model.add(Dense(1, activation="sigmoid"))
rev_model.add(Dense(1, activation="softmax"))


optimizer = Adam(learning_rate=3e-4)

rev_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
num_words
np.shape(step_train_sequences)
#step_train_sequences[0,12]

(2010, 101)

In [45]:
step_train_sequences=np.asarray(step_train_sequences).astype(np.float32)
step_train_labels=np.asarray(step_train_labels).astype(np.float32)

size = 1600
hi_train_seq = step_train_sequences[:size]
hi_train_labels = step_train_labels[:size]
hi_test_seq = step_train_sequences[size:]
hi_test_labels =  step_train_labels[size:]

history1 = rev_model.fit(
    tf.expand_dims(step_train_sequences,axis=-1),
    step_train_labels,
    epochs=20,
    validation_data=(tf.expand_dims(step_train_sequences,axis=-1),step_train_labels),
    verbose=1,
)

Epoch 1/20
63/63 [==============================] - 12s 53ms/step - loss: 0.6432 - accuracy: 0.7398 - val_loss: 0.6386 - val_accuracy: 0.7398
Epoch 2/20
63/63 [==============================] - 3s 52ms/step - loss: 0.6282 - accuracy: 0.7398 - val_loss: 0.6138 - val_accuracy: 0.7398
Epoch 3/20
63/63 [==============================] - 3s 53ms/step - loss: 0.6077 - accuracy: 0.7398 - val_loss: 0.6032 - val_accuracy: 0.7398
Epoch 4/20
63/63 [==============================] - 4s 56ms/step - loss: 0.6011 - accuracy: 0.7398 - val_loss: 0.5991 - val_accuracy: 0.7398
Epoch 5/20
63/63 [==============================] - 4s 58ms/step - loss: 0.5977 - accuracy: 0.7398 - val_loss: 0.5961 - val_accuracy: 0.7398
Epoch 6/20
63/63 [==============================] - 4s 56ms/step - loss: 0.5948 - accuracy: 0.7398 - val_loss: 0.5934 - val_accuracy: 0.7398
Epoch 7/20
63/63 [==============================] - 3s 54ms/step - loss: 0.5923 - accuracy: 0.7398 - val_loss: 0.5910 - val_accuracy: 0.7398
Epoch 8/20
6

In [46]:
print(hi_train_seq.shape)
# print(tf.expand_dims(step_train_sequences,axis=-1))
print(step_train_sequences)

(1600, 101)
[[ 0.096225    0.427025   -0.11979099 ... -0.27063     0.47540498
   0.27173406]
 [ 0.32598     0.54677    -0.37091    ... -0.205675    0.01301998
   0.25964198]
 [ 0.32598     0.54677    -0.37091    ... -0.205675    0.01301998
   0.23195446]
 ...
 [-0.289089   -0.49940002  0.287495   ... -0.283944    0.255565
   0.9840117 ]
 [-0.289089   -0.49940002  0.287495   ... -0.283944    0.255565
   0.9490345 ]
 [-0.289089   -0.49940002  0.287495   ... -0.283944    0.255565
   0.9776004 ]]


In [47]:
pred = rev_model.predict(tf.expand_dims(hi_test_seq,axis=-1))
print(pred.shape)
print(pred[0])
pred_int=[]

pred_int = pred.round().astype("int")

13/13 [==============================] - 1s 9ms/step
(410, 1)
[0.99999994]


In [48]:
print(len(hi_test_labels))
ccc=0
for i in range(len(hi_test_labels)):
    if(pred_int[i]==hi_test_labels[i]):
#         print(pred_int[i])
        ccc+=1
print(ccc)
pred

410
309


array([[0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.999

In [51]:
found_polarity=[]
# print(type(pred_int[0]))
j=0
# print(len(pred))
for j in range(len(revindex)):
    sum_1=0
    count_review=0
    average=0
    for i in range(len(pred)):

        if(ref_revarray[size+i]==j):
            count_review+=1
            sum_1+=pred_int[i]
    if(count_review==0):
        continue
    average=sum_1/count_review
    average=average.round().astype("int")
    found_polarity.append(average)

print(np.shape(found_polarity))
len(found_polarity)

(63, 1)


63

In [55]:
count=0
aa=(ref_revarray[size])
#print(len(found_polarity))
for i in range(len(found_polarity)):
    if(found_polarity[i][0]==rev_polarity[aa+i]):
        count+=1
print(count)
print(count/len(found_polarity)*100,"%")

51
80.95238095238095 %
